In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
df=pd.read_excel("C:\\Users\\CHAHAT\\Downloads\\2025-3-14-iolp-buildings.xlsx")
df.head()


In [ ]:
df.dtypes

In [ ]:
df.describe()

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.duplicated().sum()

In [ ]:
df.isnull().sum()

In [ ]:
df['Construction Date']=df['Construction Date'].fillna(df['Construction Date'].median())

In [ ]:
df.drop('Installation Name',axis=1,inplace=True)

In [ ]:
df[['Building Rentable Square Feet'	,'Available Square Feet','Construction Date']].astype(int)

In [ ]:
from datetime import datetime
current_year=datetime.now().year
df['building age']=current_year-df['Construction Date']


In [ ]:
df['vaccancy_rate']=(df['Available Square Feet']/df['Building Rentable Square Feet']*100)

In [ ]:
df['if lease']=df['Owned or Leased'].apply(lambda x: 1 if x=='L' else 0)

# ownership and location based


In [ ]:
# percenatge of property owned or leased
colors=['lightsalmon','peachpuff']
explode=(0,0)

plt.pie(df['Owned or Leased'].value_counts().values,labels=df['Owned or Leased'].value_counts().index,autopct='%1.1f%%',colors=colors,shadow=True,startangle=60,explode=explode)
plt.title('Percentage Of Property i.e owned(F) or leased(L)')
plt.show()

In [ ]:
# Which GSA Region has the highest number of properties?
high_region=df.groupby('GSA Region')['Real Property Asset Name'].count().idxmax()
high_count=df.groupby('GSA Region')['Real Property Asset Name'].count().max()
print(f'GSA Region that has the highest number of properties is {high_region} with {high_count} properties')

In [ ]:
#  properties distributed across different states and cities

property_count=df.groupby(['State','City'])['Real Property Asset Name'].count().reset_index()


In [ ]:
# Which congressional districts have the most government properties?
govt_properties=df.groupby('Congressional District')['Real Property Asset Name'].count().sort_values(ascending=False).head(10)
plt.pie(govt_properties.values,labels=govt_properties.index,autopct='%1.1f%%',shadow=True)
plt.title('top 10 Congressional districts that have the most Govt properties',fontsize=15)
plt.xlabel('congressional district number')
plt.show()

# building availability and space

In [ ]:
# Which properties have the highest available square footage?
sq_feet_property = df.groupby('Real Property Asset Name')['Available Square Feet'].sum().sort_values(ascending=False).head(20)

plt.figure(figsize=(16, 6))  
plt.bar(sq_feet_property.index, sq_feet_property.values)

plt.ylabel('Available Square Feet', fontsize=12)
plt.xlabel('Property Name', fontsize=15)
plt.title('Top 20 Properties with Highest Available Square Feet', fontsize=18)

plt.xticks(rotation=45, ha='right')  
plt.tight_layout()  

plt.show()


In [ ]:
# What is the total rentable square footage for all properties?
total_rentable_sqft = df['Building Rentable Square Feet'].sum()
print(f"Total rentable square footage for all properties: {total_rentable_sqft:,}")




In [ ]:
rentable_property = df.groupby('Real Property Asset Name')['Building Rentable Square Feet'].sum().sort_values(ascending=False).head(20)
plt.figure(figsize=(16, 6))  
plt.bar(rentable_property.index, rentable_property.values)

plt.ylabel('Rentable Sq Feet', fontsize=12)
plt.xlabel('Property Name', fontsize=15)
plt.title(' Properties with rentable square feet', fontsize=18)

plt.xticks(rotation=45, ha='right')  
plt.tight_layout()  

plt.show()

In [ ]:
# Which properties currently have available space for lease?


leased_properties = df[df['if lease'] == 1].groupby('Real Property Asset Name')['Available Square Feet'].sum()


# time and construction

In [ ]:
# What is the average age of properties based on the construction date?
average_age=df['building age'].mean()
print(f'the average age of properties is {average_age}')

In [ ]:
# Which properties were built in the last 10 years?
recent_years=df[df['Construction Date']>=(current_year-10)]
recent_properties=recent_years.sort_values(by='Construction Date',ascending=False)
recent_properties.head(10)
plt.hist(recent_properties['Construction Date'],bins=10,edgecolor='black',color='rosybrown')
plt.xlabel("last 10 years")
plt.ylabel("no of properties")
plt.title('No of properties built in  last 10 years',fontsize=14,fontweight='bold',color='crimson')
plt.tight_layout()
plt.show()


In [ ]:
# How does the construction date vary across different regions or property types?
constructed_on_regions=df.groupby(['Real Property Asset Type','GSA Region'])['Construction Date'].mean().reset_index()
colors=('darkgreen','forestgreen','seagreen')
plt.figure(figsize=(15,6))
sns.barplot(x='GSA Region',y='Construction Date',hue='Real Property Asset Type',data=constructed_on_regions,palette=colors)
plt.title('Average Construction Year by Region and Property Type',fontsize=15,fontweight='bold')
plt.xlabel('GSA Region',fontsize=15)
plt.ylabel("Average Construction Year",fontsize=15)
plt.grid(linestyle='--',alpha=0.5,axis='y')
plt.tight_layout()
plt.show()

# Building Status & Representatives


In [ ]:
# How many properties are currently active, inactive, or under construction?
properties_status=df.groupby('Building Status')['Real Property Asset Name'].count()
colors=('ghostwhite','slategrey','lightsteelblue')
explode=(0,0,0)
log_sizes = np.log1p(properties_status.values) 
plt.pie(log_sizes,labels=properties_status.index,colors=colors,autopct='%1.1f%%',shadow=True,explode=explode)
plt.tight_layout()
plt.show()


In [ ]:
# Which congressional representatives oversee the most properties?

most_properties=df.groupby('Congressional District Representative Name')['Real Property Asset Name'].value_counts().sort_values(ascending=False)

In [ ]:
# Is there a relationship between property status and ownership type (owned vs. leased)?
relation=pd.crosstab(df['Owned or Leased'],df['Building Status'])

sns.heatmap(relation,annot=True,cmap='Greens')
plt.title('Relationship between building status and ownership type',fontsize=15,fontweight='bold',color='forestgreen')
plt.show()